In [1]:
# -*- coding: utf-8 -*-
import os, glob, json, re
from collections import defaultdict
from tqdm.auto import tqdm
import pandas as pd
from transformers import pipeline

# ——— 설정 ———
FINAL_DIR  = 'combination_data'                   # JSON 파일들이 있는 폴더 경로
OUTPUT_CSV = 'output_sentiment/all_sessions_v2.csv'  # 결과 CSV 저장 경로

# 감정 분석 파이프라인
sentiment = pipeline(
    'sentiment-analysis',
    model='nlptown/bert-base-multilingual-uncased-sentiment',
    top_k=None,
    device=-1
)

def split_sentences(text):
    return re.split(r'(?<=[\.!\?])\s+', text)

def calc_speaker_emotion(content, speaker_tag):
    # 1) 해당 화자("고객"/"상담사") 대화만 추출
    lines = [
        line[len(speaker_tag)+1:].strip()
        for line in content.split('\n')
        if line.startswith(f'{speaker_tag}:')
    ]
    # 2) 문장 단위 분리
    sents = []
    for ln in lines:
        for sent in split_sentences(ln):
            if sent.strip():
                sents.append(sent)
    # 3) 별점 합산
    agg = defaultdict(float)
    for sent in tqdm(sents, desc=f'{speaker_tag} 분석', leave=False):
        scores = sentiment(sent)[0]
        for d in scores:
            star = int(d['label'][0])
            agg[star] += d['score']
    n = len(sents) or 1
    # 4) 별점별 평균
    star_scores = {f'{speaker_tag}_emo_{i}_star_score': agg[i]/n for i in range(1,6)}
    # 5) 종합 점수 & 레이블
    sent_score = sum(i * star_scores[f'{speaker_tag}_emo_{i}_star_score'] for i in range(1,6))
    if   sent_score >= 3.5: label = "긍정"
    elif sent_score >= 2.5: label = "중립"
    else:                  label = "부정"
    star_scores[f'{speaker_tag}_sent_score'] = sent_score
    star_scores[f'{speaker_tag}_sent_label'] = label
    return star_scores

# ——— 폴더 내 모든 JSON 파일 처리 ———
all_files = glob.glob(os.path.join(FINAL_DIR, '*.json'))
rows = []

for fp in tqdm(all_files, desc='세션 처리'):
    with open(fp, 'r', encoding='utf-8') as f:
        j = json.load(f)
    sid = str(j.get('session_id') or os.path.splitext(os.path.basename(fp))[0])
    # merged 포맷 or classification 포맷 구분
    if 'classification' in j:
        content = j['classification'].get('consulting_content', '')
    else:
        content = j.get('consulting_content', '')
    # 고객·상담사 감정 계산
    cust_feats = calc_speaker_emotion(content, '고객')
    agt_feats  = calc_speaker_emotion(content, '상담사')
    row = {'session_id': sid}
    row.update(cust_feats)
    row.update(agt_feats)
    rows.append(row)

# ——— DataFrame & CSV 저장 ———
df = pd.DataFrame(rows)
os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)
df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8-sig')

print(f'✅ 모든 세션 처리 완료 → {OUTPUT_CSV}')

Device set to use cpu


세션 처리:   0%|          | 0/3533 [00:00<?, ?it/s]

고객 분석:   0%|          | 0/16 [00:00<?, ?it/s]

상담사 분석:   0%|          | 0/20 [00:00<?, ?it/s]

고객 분석:   0%|          | 0/27 [00:00<?, ?it/s]

...

고객 분석:   0%|          | 0/29 [00:00<?, ?it/s]

상담사 분석:   0%|          | 0/37 [00:00<?, ?it/s]

고객 분석:   0%|          | 0/30 [00:00<?, ?it/s]

상담사 분석:   0%|          | 0/38 [00:00<?, ?it/s]

KeyboardInterrupt: 